train.csv - The training set.
Id Unique identifier for each observation.
AB-GL Fifty-six anonymized health characteristics. All are numeric except for EJ, which is categorical.
Class A binary target: 1 indicates the subject has been diagnosed with one of the three conditions, 0 indicates they have not.

[Logistic Regression, Random Forest, Gradient Boosting Models (e.g., XGBoost, LightGBM, Support Vector Machines (SVM)]

GridSearchCV
RandomizedSearchCV
Bayesian Optimization
HyperOpt
Optuna

##### Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

###### Data Loading

In [ ]:
data = pd.read_csv('train.csv')
print(data.shape)
data.head()

In [ ]:
data.info()

##### Clean data

In [ ]:
data['EJ'].value_counts()

In [ ]:
# converting categorical columns
data = data.replace({'EJ':{'A':0, 'B':1}})

In [ ]:
data.isnull().sum()

In [ ]:
# fill missing age values with the median value
data['BQ'] = data['BQ'].fillna(data['BQ'].median())
data['CB'] = data['CB'].fillna(data['CB'].median())
data['CC'] = data['CC'].fillna(data['CC'].median())
data['DU'] = data['DU'].fillna(data['DU'].median())
data['EL'] = data['EL'].fillna(data['EL'].median())

data['FC'] = data['FC'].fillna(data['FC'].median())
data['FL'] = data['FL'].fillna(data['FL'].median())
data['FS'] = data['FS'].fillna(data['FS'].median())
data['GL'] = data['GL'].fillna(data['GL'].median())

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data['Class'].value_counts()

In [ ]:
data.describe()

###### Find correlation

In [ ]:
correlation_matrix = data.corr()

correlation_threshold = 0.5

In [ ]:
# Find highly correlated features
highly_correlated_features = correlation_matrix[((correlation_matrix > correlation_threshold) | (correlation_matrix < -correlation_threshold)) & (correlation_matrix != 1)]
highly_correlated_features = highly_correlated_features.unstack().dropna().reset_index()

# Print the highly correlated features
print("Highly Correlated Features:")
for index, row in highly_correlated_features.iterrows():
    feature1 = row['level_0']
    feature2 = row['level_1']
    correlation = row[0]
    print(f"{feature1} - {feature2}: {correlation:.2f}")

In [ ]:
# find highly correlated features with diagnosis
highly_correlated_features = np.abs(correlation_matrix['Class']).sort_values(ascending=False)
highly_correlated_features= highly_correlated_features[highly_correlated_features > correlation_threshold]

# print the highly correlated features
print("Highly Correlated Features with Diagnosis: ")
for feature, correlation in highly_correlated_features.iteritems():
    print(f"{feature}: {correlation:.2f}")

## Data training and prediction analysis

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, auc, precision_recall_curve

In [ ]:
X = data.drop(['Class', 'Id'], axis=1)
y = data['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

###### Cross-validation function

In [ ]:
# Define a function to evaluate the model using cross-validation and print various metrics
def evaluate_model(model, X_train, y_train):
    cv_accuracy = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    cv_precision = cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    cv_recall = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
    cv_f1 = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    cv_roc_auc = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')

    print(f"Cross-Validation Accuracy: {np.mean(cv_accuracy)*100:.2f}")
    print(f"Cross-Validation Precision: {np.mean(cv_precision)*100:.2f}")
    print(f"Cross-Validation Recall: {np.mean(cv_recall)*100:.2f}")
    print(f"Cross-Validation F1-Score: {np.mean(cv_f1)*100:.2f}")
    print(f"Cross-Validation ROC-AUC: {np.mean(cv_roc_auc)*100:.2f}")

    return cv_accuracy, cv_precision, cv_recall, cv_f1, cv_roc_auc

### Without Cross-validation

#### Unscaled data

##### Test 1

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100}")

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot the learning curves for the best model
plt.figure(figsize=(8, 6))
train_sizes, train_scores, val_scores = learning_curve(
    estimator=model, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5, scoring='accuracy', shuffle=True, random_state=42
)

# Calculate the mean and standard deviation of train and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)

# Plot the learning curves with the mean and standard deviation
plt.plot(train_sizes, train_scores_mean, label='Training Accuracy', color='blue')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_scores_mean, label='Validation Accuracy', color='red')
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.2, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()

##### Test 2

#### Hyperparameter tuning using GridSearchCV

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {'var_smoothing': np.logspace(0, -9, num=100)}

# Create a Naive Bayes classifier
model = GaussianNB()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the test data using the best model
y_pred = best_model.predict(X_test)

# print the best parameters found during hyperparameter tuning
print("\nBest Parameters: ")
print(best_params)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100}")

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot the learning curves for the best model
plt.figure(figsize=(8, 6))
train_sizes, train_scores, val_scores = learning_curve(
    estimator=best_model, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5, scoring='accuracy', shuffle=True, random_state=42
)

# Calculate the mean and standard deviation of train and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)

# Plot the learning curves with the mean and standard deviation
plt.plot(train_sizes, train_scores_mean, label='Training Accuracy', color='blue')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_scores_mean, label='Validation Accuracy', color='red')
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.2, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()

#### Scaled data

##### Test 1

In [ ]:
model = GaussianNB()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100}")

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot the learning curves for the best model
plt.figure(figsize=(8, 6))
train_sizes, train_scores, val_scores = learning_curve(
    estimator=model, X=X_train_scaled, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5, scoring='accuracy', shuffle=True, random_state=42
)

# Calculate the mean and standard deviation of train and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)

# Plot the learning curves with the mean and standard deviation
plt.plot(train_sizes, train_scores_mean, label='Training Accuracy', color='blue')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_scores_mean, label='Validation Accuracy', color='red')
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.2, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()

##### Test 2

#### Hyperparameter tuning using GridSearchCV

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {'var_smoothing': np.logspace(0, -9, num=100)}

# Create a Naive Bayes classifier
model = GaussianNB()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the test data using the best model
y_pred = best_model.predict(X_test_scaled)

# print the best parameters found during hyperparameter tuning
print("\nBest Parameters: ")
print(best_params)

print(f"Accuracy: {accuracy*100}")

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot the learning curves for the best model
plt.figure(figsize=(8, 6))
train_sizes, train_scores, val_scores = learning_curve(
    estimator=best_model, X=X_train_scaled, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5, scoring='accuracy', shuffle=True, random_state=42
)

# Calculate the mean and standard deviation of train and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)

# Plot the learning curves with the mean and standard deviation
plt.plot(train_sizes, train_scores_mean, label='Training Accuracy', color='blue')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_scores_mean, label='Validation Accuracy', color='red')
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.2, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()

#### With cross-validation

#### Unscaled-data

##### Test 1

In [ ]:
# Test 1: Logistic Regression without scaling
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model using cross-validation and print various metrics
print("Test 1: Support Vector Machines without scaling")
cv_accuracy, cv_precision, cv_recall, cv_f1, cv_roc_auc = evaluate_model(model, X_train, y_train)

# Plot the precision-recall curve
y_probs = model.decision_function(X_test)
precision, recall, _ = precision_recall_curve(y_test, y_probs)
plt.plot(recall, precision, color='b', label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

##### Test 2

#### Hyperparameter tuning using GridSearchCV

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {'var_smoothing': np.logspace(0, -9, num=100)}

# Create a Naive Bayes classifier
model = GaussianNB()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the test data using the best model
y_pred = best_model.predict(X_test)

# print the best parameters found during hyperparameter tuning
print("\nBest Parameters: ")
print(best_params)

# Evaluate the model using cross-validation and print various metrics
print("Test 1: Gaussian Naive Bayes without scaling")
cv_accuracy, cv_precision, cv_recall, cv_f1, cv_roc_auc = evaluate_model(best_model, X_train, y_train)

# Plot the precision-recall curve
y_probs = model.decision_function(X_test)
precision, recall, _ = precision_recall_curve(y_test, y_probs)
plt.plot(recall, precision, color='b', label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Plot the learning curves for the best model
plt.figure(figsize=(8, 6))
train_sizes, train_scores, val_scores = learning_curve(
    estimator=model, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5, scoring='accuracy', shuffle=True, random_state=42
)

# Calculate the mean and standard deviation of train and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)

# Plot the learning curves with the mean and standard deviation
plt.plot(train_sizes, train_scores_mean, label='Training Accuracy', color='blue')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_scores_mean, label='Validation Accuracy', color='red')
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.2, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()

#### Scaled-data

##### Test 1

In [ ]:
# Test 1: Logistic Regression with scaling
model = GaussianNB()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# Evaluate the model using cross-validation and print various metrics
print("Test 2: Support Vector Machines with scaling")
cv_accuracy, cv_precision, cv_recall, cv_f1, cv_roc_auc = evaluate_model(model, X_train_scaled, y_train)

# Plot the precision-recall curve
y_probs = model.decision_function(X_test_scaled)
precision, recall, _ = precision_recall_curve(y_test, y_probs)
plt.plot(recall, precision, color='b', label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

##### Test 2

#### Hyperparameter tuning using GridSearchCV

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {'var_smoothing': np.logspace(0, -9, num=100)}

# Create a Naive Bayes classifier
model = GaussianNB()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the test data using the best model
y_pred = best_model.predict(X_test_scaled)

# print the best parameters found during hyperparameter tuning
print("\nBest Parameters: ")
print(best_params)

# Evaluate the model using cross-validation and print various metrics
print("Test 2: Gaussian Naive Bayes with scaling")
cv_accuracy, cv_precision, cv_recall, cv_f1, cv_roc_auc = evaluate_model(best_model, X_train_scaled, y_train)

# Plot the precision-recall curve
y_probs = model.decision_function(X_test_scaled)
precision, recall, _ = precision_recall_curve(y_test, y_probs)
plt.plot(recall, precision, color='b', label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Plot the learning curves for the best model
plt.figure(figsize=(8, 6))
train_sizes, train_scores, val_scores = learning_curve(
    estimator=model, X=X_train_scaled, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5, scoring='accuracy', shuffle=True, random_state=42
)

# Calculate the mean and standard deviation of train and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)

# Plot the learning curves with the mean and standard deviation
plt.plot(train_sizes, train_scores_mean, label='Training Accuracy', color='blue')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_scores_mean, label='Validation Accuracy', color='red')
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.2, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()